In [ ]:
#|default_exp distributed.fugue

# Fugue

> Distributed fugue backend

In [ ]:
#|export
import copy
import tempfile
from pathlib import Path
from typing import Any, Callable, Iterable, List, Optional

import cloudpickle
import lightgbm as lgb
import pandas as pd
from fugue import transform
from pyspark.ml.feature import VectorAssembler

from mlforecast.core import (
    DateFeature,
    Differences,
    Freq,
    LagTransforms,
    Lags,
    TimeSeries,
    _name_models,
)

In [ ]:
#|export
class FugueMLForecast:
    def __init__(
        self,
        models,
        freq: Optional[Freq] = None,
        lags: Optional[Lags] = None,
        lag_transforms: Optional[LagTransforms] = None,
        date_features: Optional[Iterable[DateFeature]] = None,
        differences: Optional[Differences] = None,
        num_threads: int = 1,
        engine = None,
    ):
        if not isinstance(models, dict) and not isinstance(models, list):
            models = [models]
        if isinstance(models, list):
            model_names = _name_models([m.__class__.__name__ for m in models])
            models_with_names = dict(zip(model_names, models))
        else:
            models_with_names = models
        self.models = models_with_names
        self._base_ts = TimeSeries(
            freq, lags, lag_transforms, date_features, differences, num_threads
        )
        self.engine = engine
        
    def __repr__(self) -> str:
        return (
            f'{self.__class__.__name__}(models=[{", ".join(self.models.keys())}], '
            f"freq={self._base_ts.freq}, "
            f"lag_features={list(self._base_ts.transforms.keys())}, "
            f"date_features={self._base_ts.date_features}, "
            f"num_threads={self._base_ts.num_threads}, "
            f"engine={self.engine})"
        )

    @staticmethod
    def _preprocess_partition(
        part: pd.DataFrame,
        base_ts: TimeSeries,        
        id_col: str,
        time_col: str,
        target_col: str,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,        
    ) -> List[List[Any]]:
        ts = copy.deepcopy(base_ts)
        transformed = ts.fit_transform(
            part,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
        )
        return [[cloudpickle.dumps(ts), cloudpickle.dumps(transformed)]]
    
    @staticmethod
    def _retrieve_df(items: List[List[Any]]) -> Iterable[pd.DataFrame]:
        for _, serialized_df in items:
            yield cloudpickle.loads(serialized_df)

    def preprocess(
        self,
        data,
        id_col: str,
        time_col: str,
        target_col: str,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
    ):
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col
        self.partition_results = transform(
            data,
            FugueMLForecast._preprocess_partition,
            params={
                'base_ts': self._base_ts,
                'id_col': id_col,
                'time_col': time_col,
                'target_col': target_col,
                'static_features': static_features,
                'dropna': dropna,
                'keep_last_n': keep_last_n,
            },
            schema='ts:binary,df:binary',
            engine=self.engine,
        )
        base_schema = f'{id_col}:string,{time_col}:datetime,{target_col}:double'
        features_dtypes = [f'{feat}:double' for feat in self._base_ts.features]        
        schema = base_schema + ',' + ','.join(features_dtypes)
        return transform(
            self.partition_results,
            FugueMLForecast._retrieve_df,
            schema=schema,
            engine=self.engine,
        )
    
    def fit(
        self,
        data,
        id_col: str,
        time_col: str,
        target_col: str,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
    ):
        preprocessed = self.preprocess(
            data,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
        )
        feature_cols = [x for x in preprocessed.columns if x not in (id_col, time_col, target_col)]
        featurizer = VectorAssembler(inputCols=feature_cols, outputCol="features")
        train_data = featurizer.transform(preprocessed)[target_col, "features"]
        for name, model in self.models.items():
            trained_model = model.setLabelCol(target_col).fit(train_data)
            # horrible way to get a lightgbm booster, please look away
            with tempfile.TemporaryDirectory() as tmp_dir:
                trained_model.saveNativeModel(tmp_dir)
                tmp_path = Path(tmp_dir)
                txt_file = next(tmp_path.glob('*.txt'))
                bst = lgb.Booster(model_file=txt_file)
            self.models[name] = bst
        return self

    @staticmethod
    def _predict(
        items: List[List[Any]],
        models,        
        horizon,
        dynamic_dfs,
        before_predict_callback,
        after_predict_callback,
    ) -> Iterable[pd.DataFrame]:
        for serialized_ts, _ in items:
            ts = cloudpickle.loads(serialized_ts)
            res = ts.predict(
                models=models,
                horizon=horizon,
                dynamic_dfs=dynamic_dfs,
                before_predict_callback=before_predict_callback,
                after_predict_callback=after_predict_callback,
            )
            yield res.reset_index()
            
    def predict(
        self,
        horizon: int,
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,
        before_predict_callback: Optional[Callable] = None,
        after_predict_callback: Optional[Callable] = None,
    ):
        model_names = self.models.keys()
        models_schema = ','.join(f'{model_name}:double' for model_name in model_names)
        schema = f'{self.id_col}:string,{self.time_col}:datetime,' + models_schema
        return transform(
            self.partition_results,
            FugueMLForecast._predict,
            params={
                'models': self.models,
                'horizon': horizon,
                'dynamic_dfs': dynamic_dfs,
                'before_predict_callback': before_predict_callback,
                'after_predict_callback': after_predict_callback,
            },
            schema=schema,
            engine=self.engine,
        )

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

from pyspark.sql import SparkSession
from window_ops.expanding import expanding_mean

from mlforecast.utils import generate_daily_series

In [ ]:
spark = (
    SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.10.2")
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
    .getOrCreate()
)

In [ ]:
series = generate_daily_series(100).reset_index()
series['unique_id'] = series['unique_id'].astype(str)
spark_series = spark.createDataFrame(series).repartitionByRange(4, 'unique_id')

In [ ]:
from synapse.ml.lightgbm import LightGBMRegressor

In [ ]:
fcst = FugueMLForecast(
    [LightGBMRegressor()],
    freq='D',
    lags=[1],
    lag_transforms={
        1: [expanding_mean]
    },
    date_features=['dayofweek'],
    engine=spark,
)

In [ ]:
fcst.fit(spark_series, id_col='unique_id', time_col='ds', target_col='y')

FugueMLForecast(models=[LightGBMRegressor], freq=<Day>, lag_features=['lag1', 'expanding_mean_lag1'], date_features=['dayofweek'], num_threads=1, engine=<pyspark.sql.session.SparkSession object>)

In [ ]:
fcst.predict(10).toPandas()

,unique_id,ds,LightGBMRegressor
0,id_00,2000-08-10,5.263606
1,id_00,2000-08-11,6.255906
2,id_00,2000-08-12,0.263115
3,id_00,2000-08-13,1.258064
4,id_00,2000-08-14,2.249121
...,...,...,...
995,id_99,2000-07-05,2.249115
996,id_99,2000-07-06,3.241592
997,id_99,2000-07-07,4.246263
998,id_99,2000-07-08,5.260917
